# Скачиваем данные и устанавливаем программы

In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/gms2.lst

--2022-10-19 13:31:30--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220305 (215K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 215.14K  --.-KB/s    in 0.003s  

2022-10-19 13:31:30 (67.6 MB/s) - ‘gms2.lst’ saved [220305/220305]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw1/main/data/scaffolds.fasta

--2022-10-19 13:31:32--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw1/main/data/scaffolds.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3971948 (3.8M) [text/plain]
Saving to: ‘scaffolds.fasta’

scaffolds.fasta     100%[===================>]   3.79M  --.-KB/s    in 0.01s   

2022-10-19 13:31:32 (264 MB/s) - ‘scaffolds.fasta’ saved [3971948/3971948]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/proteins.fasta

--2022-10-19 13:31:34--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1403607 (1.3M) [text/plain]
Saving to: ‘proteins.fasta’

proteins.fasta      100%[===================>]   1.34M  --.-KB/s    in 0.009s  

2022-10-19 13:31:34 (155 MB/s) - ‘proteins.fasta’ saved [1403607/1403607]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt

--2022-10-19 13:31:39--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870940 (851K) [text/plain]
Saving to: ‘scaffolds.hits_from_MIL_1.txt’

scaffolds.hits_from 100%[===================>] 850.53K  --.-KB/s    in 0.007s  

2022-10-19 13:31:39 (117 MB/s) - ‘scaffolds.hits_from_MIL_1.txt’ saved [870940/870940]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-19 13:31:42--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66190 (65K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt’

scaffolds.hits_from 100%[===================>]  64.64K  --.-KB/s    in 0.001s  

2022-10-19 13:31:42 (49.6 MB/s) - ‘scaffolds.hits_from_SwissProt.txt’ saved [66190/66190]



## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Аннотация генома

In [167]:
from Bio import SeqIO

scaffolds = dict() 

for record in SeqIO.parse("/content/scaffolds.fasta", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = "19-OCT-2022"
  scaffolds[record.id] = record

In [168]:
print(scaffolds["scaffold1_cov231"])

ID: scaffold1_cov231
Name: scaffold1_cov231
Description: scaffold1_cov231
Number of features: 0
/molecule_type=DNA
/date=19-OCT-2022
Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA')


## Добавляем координаты генов и трансляции генов

In [169]:
from Bio.SeqFeature import SeqFeature, FeatureLocation

for record in SeqIO.parse("/content/proteins.fasta", "fasta"):
  gene_description = record.description.split() # id, scaffold, start, end, strand, ...
  id = gene_description[0]
  scaffold = gene_description[1]
  start = int(gene_description[2])
  end = int(gene_description[3])
  strand = 1 if gene_description[4] == '+' else -1
  
  feature = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feature.qualifiers['locus_tag'] = [id]
  feature.qualifiers['translation'] = [record.seq]
  scaffolds[scaffold].features.append(feature)

In [170]:
print(scaffolds["scaffold1_cov231"])

ID: scaffold1_cov231
Name: scaffold1_cov231
Description: scaffold1_cov231
Number of features: 3536
/molecule_type=DNA
/date=19-OCT-2022
Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA')


In [171]:
print(scaffolds["scaffold1_cov231"].features[0])

type: CDS
location: [3:299](-)
qualifiers:
    Key: locus_tag, Value: ['1']
    Key: translation, Value: [Seq('MPKYTKPRKTWQYSNEFKVKAVQLSLIEGIQVKEVAVTLDIHPFMLSRWRKEYR...LKK')]



## Добавляем функции белков (из бактерии MIL-1)

In [172]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/scaffolds.hits_from_MIL_1.txt", sep='\t', header=None, names=
                 ["Query sequence ID", "Subject sequence ID", "Percentage of identical positions", 
                  "Alignment length", "Number of mismatches", "Number of gap openings",
                  "Start position of alignment on query sequence", "End position of alignment on query sequence",
                  "Start position of alignment on subject sequence", "End position of alignment on subject sequence",
                  "Expect value", "Bit score"])
df = df[pd.DataFrame([line for line in df['Subject sequence ID'].str.split('_').values]).loc[:, 2].str[0:3] == 'CCU']
df = df.sort_values(by=["Bit score"], ascending=False).drop_duplicates(subset=["Query sequence ID"]).sort_values(by=["Query sequence ID"])
df.head()

,Query sequence ID,Subject sequence ID,Percentage of identical positions,Alignment length,Number of mismatches,Number of gap openings,Start position of alignment on query sequence,End position of alignment on query sequence,Start position of alignment on subject sequence,End position of alignment on subject sequence,Expect value,Bit score
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
7,3,lcl|HF680312.1_prot_CCU71928.1_1479,100.000,173,0,0,1,173,1,173,1.660000e-130,359.0
8,5,lcl|HF680312.1_prot_CCU71924.1_1475,97.143,35,1,0,1,35,1,35,1.510000e-19,75.1
9,6,lcl|HF680312.1_prot_CCU71587.1_1138,98.024,253,5,0,1,253,29,281,1.320000e-176,484.0
11,7,lcl|HF680312.1_prot_CCU71586.1_1137,100.000,391,0,0,1,391,1,391,0.000000e+00,810.0


In [173]:
def print_dict(d, count=5):
  ind = 0
  for key, value in d.items():
    if ind < count:
      print(key, value)
      ind += 1

In [174]:
# создадим словарь соответствия Query sequence ID -> Subject sequence ID
qs_id_ss_id = dict()
for _, record in df.iterrows():
  qs_id_ss_id[record['Query sequence ID']] = record['Subject sequence ID'].split('_')[2]

print_dict(qs_id_ss_id)

1 CCU71653.1
3 CCU71928.1
5 CCU71924.1
6 CCU71587.1
7 CCU71586.1


In [175]:
mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [176]:
# создадим словарь соответствия Subject sequence ID -> product(функция белка)
ss_id_product = dict()
for mil_1_feature in mil_1_genome.features:
  if ('protein_id' not in mil_1_feature.qualifiers) or ('product' not in mil_1_feature.qualifiers):
    continue
  ss_id_product[mil_1_feature.qualifiers['protein_id'][0]] = mil_1_feature.qualifiers['product']

print_dict(ss_id_product)

CCU70450.1 ['chromosomal replication initiator protein']
CCU70451.1 ['DNA polymerase III subunit beta']
CCU70452.1 ['DNA replication and repair protein RecF']
CCU70453.1 ['DNA gyrase subunit B']
CCU70454.1 ['hypothetical protein']


In [177]:
# будем итерироваться по скаффолдам, для каждого скаффолда - по найденным генам 
# если для гена найден аналог в геноме похожей бактерии, добавляем соответствующее название
# если для гена найден аналог в геноме похожей бактерии и у него указана функция белка, добавляем соответствующую функцию белка
count_added_notes = 0
count_added_products = 0
for scaffold_name, scaffold in scaffolds.items():
  for feature in scaffold.features:
    qs_id = int(feature.qualifiers['locus_tag'][0])
    if qs_id in qs_id_ss_id:
      feature.qualifiers['note'] = [qs_id_ss_id[qs_id]]
      count_added_notes += 1
      ss_id = qs_id_ss_id[qs_id]
      if ss_id in ss_id_product:
        feature.qualifiers['product'] = ss_id_product[ss_id]
        count_added_products += 1
print("Добавлено названий", count_added_notes)
print("Добавлено функций", count_added_products)

Добавлено названий 3332
Добавлено функций 3332


In [178]:
print(scaffolds["scaffold1_cov231"].features[0])

type: CDS
location: [3:299](-)
qualifiers:
    Key: locus_tag, Value: ['1']
    Key: note, Value: ['CCU71653.1']
    Key: product, Value: ['transposase orfA']
    Key: translation, Value: [Seq('MPKYTKPRKTWQYSNEFKVKAVQLSLIEGIQVKEVAVTLDIHPFMLSRWRKEYR...LKK')]



## Добавляем функции белков (из БД SwissProt)

In [179]:
df = pd.read_csv("/content/scaffolds.hits_from_SwissProt.txt", sep='\t', header=None, names=
                 ["Query sequence ID", "Subject sequence ID", "Percentage of identical positions", 
                  "Alignment length", "Number of mismatches", "Number of gap openings",
                  "Start position of alignment on query sequence", "End position of alignment on query sequence",
                  "Start position of alignment on subject sequence", "End position of alignment on subject sequence",
                  "Expect value", "Bit score"])
df = df.sort_values(by=["Bit score"], ascending=False).drop_duplicates(subset=["Query sequence ID"]).sort_values(by=["Query sequence ID"])
df.head()

,Query sequence ID,Subject sequence ID,Percentage of identical positions,Alignment length,Number of mismatches,Number of gap openings,Start position of alignment on query sequence,End position of alignment on query sequence,Start position of alignment on subject sequence,End position of alignment on subject sequence,Expect value,Bit score
0,169,sp|P55546|Y4LF_SINFN,42.553,141,80,1,1,141,1,140,1.180000e-36,144.0
1,428,sp|Q06916|GUFA_MYXXA,43.802,242,136,0,71,312,13,254,2.330000e-47,162.0
69,591,sp|P08956|T1RK_ECOLI,40.701,1199,642,23,8,1171,5,1169,0.000000e+00,837.0
74,597,sp|Q9V1E9|KTHY_PYRAB,39.073,151,76,5,3,138,4,153,3.120000e-15,73.9
78,600,sp|P06990|T1SB_ECOLX,35.794,447,271,5,4,436,19,463,4.330000e-71,238.0


In [180]:
# создадим словарь соответствия Query sequence ID -> Subject sequence ID
qs_id_ss_id = dict()
for _, record in df.iterrows():
  qs_id_ss_id[record['Query sequence ID']] = record['Subject sequence ID'].split('|')[-1]

print_dict(qs_id_ss_id)

169 Y4LF_SINFN
428 GUFA_MYXXA
591 T1RK_ECOLI
597 KTHY_PYRAB
600 T1SB_ECOLX


Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [181]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2022-10-19 14:59:42--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz.1’

uniprot_sprot.dat.g 100%[===================>] 612.35M   149MB/s    in 3.7s    

2022-10-19 14:59:46 (166 MB/s) - ‘uniprot_sprot.dat.gz.1’ saved [642093634/642093634]



In [182]:
!gzip -d uniprot_sprot.dat.gz

gzip: uniprot_sprot.dat already exists; do you wish to overwrite (y or n)? y


In [183]:
!grep -E '^(ID|DE   RecName: Full=)' uniprot_sprot.dat > uniprot_lines.txt

In [185]:
# создадим словарь соответствия Subject sequence ID -> product(функция белка)
ss_id_product = dict()

id = ''

with open('uniprot_lines.txt') as file:
  for line in file:
    tokens = line.split()
    if tokens[0] == 'ID':
      id = tokens[1]
    else:
      ss_id_product[id] = line.split('=')[-1][:-2]

print_dict(ss_id_product)

001R_FRG3G Putative transcription factor 001R
002L_FRG3G Uncharacterized protein 002L
002R_IIV3 Uncharacterized protein 002R
003L_IIV3 Uncharacterized protein 003L
003R_FRG3G Uncharacterized protein 3R


In [186]:
# будем итерироваться по скаффолдам, для каждого скаффолда - по найденным генам 
# если для гена найден аналог в геноме похожей бактерии, добавляем соответствующее название
# если для гена найден аналог в геноме похожей бактерии и у него указана функция белка, добавляем соответствующую функцию белка
count_added_notes = 0
count_added_products = 0
for scaffold_name, scaffold in scaffolds.items():
  for feature in scaffold.features:
    qs_id = int(feature.qualifiers['locus_tag'][0])
    if qs_id in qs_id_ss_id:
      feature.qualifiers['note'] = [qs_id_ss_id[qs_id]]
      count_added_notes += 1
      ss_id = qs_id_ss_id[qs_id]
      if ss_id in ss_id_product:
        feature.qualifiers['product'] = ss_id_product[ss_id]
        count_added_products += 1
print("Добавлено названий", count_added_notes)
print("Добавлено функций", count_added_products)

Добавлено названий 53
Добавлено функций 53


In [187]:
# записываем данные в файл
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

65